## Imports

In [1]:
from geopy.geocoders import GoogleV3
import numpy as np
import geocoder
import json
import pandas as pd
from geojson import Feature, FeatureCollection, Point
import tabula
from tabula.io import read_pdf
import re
import os

## Import data from PDF

In [2]:
# Gotta figure out how to get rid of the java error.
# SO: https://stackoverflow.com/questions/54817211/java-command-is-not-found-from-this-python-process-please-ensure-java-is-inst

dsf = tabula.read_pdf('condo.pdf', pages='all',lattice=True)

## Pandas Manipulation

In [3]:
df = dsf[0]
df.columns = df.iloc[0]
df = df.drop(df.index[0])
### This next line gets ride of the Nan ###
df = df.iloc[: , 1:]
df = df.replace(np.nan, 'Not Available', regex=True)
df = df.rename(columns={'Building / Address / City': 'building_address_city','Days on\rMarket':'days_on_market',\
                        'Price /\rSq. Ft.':'price_per_sqft'})
df['geo_address'] = df['building_address_city']
df['geo_address'] = df['geo_address'].map(lambda x: re.sub(r'\r', ' ', x))
df = df.replace('\n',' ',regex=True)
df2 = df['geo_address'].str.extract(r'(?P<building_name>\D*)(?P<address_only>\d.*)', expand=True)

Review this later (it's about how to properly merge DFs): https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

In [4]:
result = pd.merge(df,df2,left_index=True, right_index=True)
df = result

In [5]:
df

,building_address_city,List Date,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address,building_name,address_only
1,Residences by Armani Casa\r18975 Collins Ave 3...,11/16/2021,8/22/2022,246,"$5,500,000.00",3255,"$1,689.71",Viviane Wolak,One Sotheby's Int'l Realty,Luciene Cofresi PA,Fortune International Realty,Residences by Armani Casa 18975 Collins Ave 30...,Residences by Armani Casa,18975 Collins Ave 3000 Sunny Isles Beach
2,Four Seasons Residences\r1425 Brickell Ave 66E...,6/20/2022,8/23/2022,19,"$3,200,000.00",2738,"$1,168.74",Philippe Diener,Brown Harris Stevens,Mariana Garber PA,Coldwell Banker Realty,Four Seasons Residences 1425 Brickell Ave 66E ...,Four Seasons Residences,1425 Brickell Ave 66E Miami
3,Portovita North\r20155 NE 38th Ct 1102\rAventura,2/10/2022,8/23/2022,134,"$2,650,000.00",3374,$785.42,Natalie Abadi,OWW Realty,Sylvia Fragos,Great Estates Int'l Realty,Portovita North 20155 NE 38th Ct 1102 Aventura,Portovita North,20155 NE 38th Ct 1102 Aventura
4,Midtown 2\r3470 E Coast Ave PH207\rMiami,1/20/2022,8/24/2022,71,"$2,200,000.00",2928,$751.37,Jacob Zebede,KZ Commercial Real Estate,Silvio Islamaj,Keller Williams Eagle Realty,Midtown 2 3470 E Coast Ave PH207 Miami,Midtown,2 3470 E Coast Ave PH207 Miami
5,Santa Maria\r1643 Brickell Ave 2603\rMiami,5/31/2022,8/22/2022,14,"$2,000,000.00",2180,$917.43,Fabian Dominguez,Fortune International Realty,Bethania Padron,Edivial International Realty,Santa Maria 1643 Brickell Ave 2603 Miami,Santa Maria,1643 Brickell Ave 2603 Miami
6,Grove Hill Tower\r2645 S Bayshore Dr 1001\rMiami,5/11/2022,8/26/2022,2,"$1,800,000.00",1910,$942.41,Jamie Jessic Ginsberg,BHHS EWM Realty,Raquel Patterson,Luxury Living Realty,Grove Hill Tower 2645 S Bayshore Dr 1001 Miami,Grove Hill Tower,2645 S Bayshore Dr 1001 Miami
7,Oceania\r16400 Collins Ave 1646\rSunny Isles B...,7/22/2022,8/24/2022,7,"$1,670,000.00",2340,$713.68,Stacey Legro,"Oceania Brokerage, Inc.",Zaida B PA Rizzo,"Interinvestments Realty, Inc.",Oceania 16400 Collins Ave 1646 Sunny Isles Beach,Oceania,16400 Collins Ave 1646 Sunny Isles Beach
8,Four Seasons Residences\r1425 Brickell Ave 66D...,3/16/2022,8/23/2022,115,"$1,600,000.00",1732,$923.79,Philippe Diener,Brown Harris Stevens,Mariana Garber PA,Coldwell Banker Realty,Four Seasons Residences 1425 Brickell Ave 66D ...,Four Seasons Residences,1425 Brickell Ave 66D Miami
9,1010 Brickell\r1010 Brickell Ave 2301\rMiami,3/29/2022,8/25/2022,118,"$1,499,000.00",1753,$855.11,Ana Teresa Rodriguez,Coldwell Banker Realty,David Jimenez PA,BHHS EWM Realty,1010 Brickell 1010 Brickell Ave 2301 Miami,,1010 Brickell 1010 Brickell Ave 2301 Miami
10,S0outh Pointe Towers\r400 S Pointe Dr 906\rMia...,6/1/2022,8/25/2022,20,"$1,435,000.00",1042,"$1,377.16",John Lennon,South Pointe Drive Realty Inc,Adi Zilberberg,Gary Hennes Realtors,S0outh Pointe Towers 400 S Pointe Dr 906 Miami...,S,0outh Pointe Towers 400 S Pointe Dr 906 Miami ...


In [6]:
df.columns

Index(['building_address_city', 'List Date', 'Close Date', 'days_on_market',
       'Sale Price', 'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker',
       'Buyer Agent', 'Buyer Broker', 'geo_address', 'building_name',
       'address_only'],
      dtype='object')

## Geocoder Setup and Execution

In [7]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [8]:
df['loc'] = df['address_only'].apply(geolocator.geocode, timeout=10)

In [9]:
df['loc']

1     (18975 Collins Ave #3000, Sunny Isles Beach, F...
2     (1425 Brickell Ave #66e, Miami, FL 33131, USA,...
3     (20155 NE 38th Ct APT 1102, Aventura, FL 33180...
4     (3470 E Coast Ave, Miami, FL 33137, USA, (25.8...
5     (1643 Brickell Ave APT 2603, Miami, FL 33129, ...
6     (2645 S Bayshore Dr APT 1001, Coconut Grove, F...
7     (16400 Collins Ave APT 1646, Sunny Isles Beach...
8     (1425 Brickell Ave #66d, Miami, FL 33131, USA,...
9     (1010 Brickell, 1010 Brickell Ave #2301, Miami...
10    (400 South Pointe Dr APT 906, Miami Beach, FL ...
Name: loc, dtype: object

In [10]:
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)

Columns must be same length as key explained
- https://cumsum.wordpress.com/2021/06/01/pandas-valueerror-columns-must-be-same-length-as-key/

Creating `df2` based on `df` row indicies.

https://datascience.stackexchange.com/questions/77033/creating-a-new-dataframe-with-specific-row-numbers-from-another

What's tripping this error is when the lat/lon/alititude generator is fed a Nan value to parse - it's effectively trying to split a `null` value into three separate parts - which is impossible.

The code below effectively caputres the anatomy of this error. It creates a new `df` based on a list of indicies from an existing one. Row `56` has no `point` since the `geocoder` couldn't find coordinates for it. The other three do. Splitting the `point` column fails only when row `56` - with `null` values - is attempted.
```
df2 = df.iloc[[56,57,58,59],:]
df2[['lat','lon','altitude']] = pd.DataFrame(df2['point'].to_list(),index=df2.index)
```

In [13]:
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

df

,building_address_city,List Date,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address,building_name,address_only,loc,point,lat,lon,altitude
1,Residences by Armani Casa\r18975 Collins Ave 3...,11/16/2021,8/22/2022,246,"$5,500,000.00",3255,"$1,689.71",Viviane Wolak,One Sotheby's Int'l Realty,Luciene Cofresi PA,Fortune International Realty,Residences by Armani Casa 18975 Collins Ave 30...,Residences by Armani Casa,18975 Collins Ave 3000 Sunny Isles Beach,"(18975 Collins Ave #3000, Sunny Isles Beach, F...","(25.9521284, -80.11970810000001, 0.0)",25.952128,-80.119708,0.0
2,Four Seasons Residences\r1425 Brickell Ave 66E...,6/20/2022,8/23/2022,19,"$3,200,000.00",2738,"$1,168.74",Philippe Diener,Brown Harris Stevens,Mariana Garber PA,Coldwell Banker Realty,Four Seasons Residences 1425 Brickell Ave 66E ...,Four Seasons Residences,1425 Brickell Ave 66E Miami,"(1425 Brickell Ave #66e, Miami, FL 33131, USA,...","(25.7591323, -80.1918295, 0.0)",25.759132,-80.191829,0.0
3,Portovita North\r20155 NE 38th Ct 1102\rAventura,2/10/2022,8/23/2022,134,"$2,650,000.00",3374,$785.42,Natalie Abadi,OWW Realty,Sylvia Fragos,Great Estates Int'l Realty,Portovita North 20155 NE 38th Ct 1102 Aventura,Portovita North,20155 NE 38th Ct 1102 Aventura,"(20155 NE 38th Ct APT 1102, Aventura, FL 33180...","(25.9625239, -80.1257694, 0.0)",25.962524,-80.125769,0.0
4,Midtown 2\r3470 E Coast Ave PH207\rMiami,1/20/2022,8/24/2022,71,"$2,200,000.00",2928,$751.37,Jacob Zebede,KZ Commercial Real Estate,Silvio Islamaj,Keller Williams Eagle Realty,Midtown 2 3470 E Coast Ave PH207 Miami,Midtown,2 3470 E Coast Ave PH207 Miami,"(3470 E Coast Ave, Miami, FL 33137, USA, (25.8...","(25.809553, -80.1915927, 0.0)",25.809553,-80.191593,0.0
5,Santa Maria\r1643 Brickell Ave 2603\rMiami,5/31/2022,8/22/2022,14,"$2,000,000.00",2180,$917.43,Fabian Dominguez,Fortune International Realty,Bethania Padron,Edivial International Realty,Santa Maria 1643 Brickell Ave 2603 Miami,Santa Maria,1643 Brickell Ave 2603 Miami,"(1643 Brickell Ave APT 2603, Miami, FL 33129, ...","(25.755259, -80.1947467, 0.0)",25.755259,-80.194747,0.0
6,Grove Hill Tower\r2645 S Bayshore Dr 1001\rMiami,5/11/2022,8/26/2022,2,"$1,800,000.00",1910,$942.41,Jamie Jessic Ginsberg,BHHS EWM Realty,Raquel Patterson,Luxury Living Realty,Grove Hill Tower 2645 S Bayshore Dr 1001 Miami,Grove Hill Tower,2645 S Bayshore Dr 1001 Miami,"(2645 S Bayshore Dr APT 1001, Coconut Grove, F...","(25.732054, -80.235655, 0.0)",25.732054,-80.235655,0.0
7,Oceania\r16400 Collins Ave 1646\rSunny Isles B...,7/22/2022,8/24/2022,7,"$1,670,000.00",2340,$713.68,Stacey Legro,"Oceania Brokerage, Inc.",Zaida B PA Rizzo,"Interinvestments Realty, Inc.",Oceania 16400 Collins Ave 1646 Sunny Isles Beach,Oceania,16400 Collins Ave 1646 Sunny Isles Beach,"(16400 Collins Ave APT 1646, Sunny Isles Beach...","(25.9283753, -80.1238589, 0.0)",25.928375,-80.123859,0.0
8,Four Seasons Residences\r1425 Brickell Ave 66D...,3/16/2022,8/23/2022,115,"$1,600,000.00",1732,$923.79,Philippe Diener,Brown Harris Stevens,Mariana Garber PA,Coldwell Banker Realty,Four Seasons Residences 1425 Brickell Ave 66D ...,Four Seasons Residences,1425 Brickell Ave 66D Miami,"(1425 Brickell Ave #66d, Miami, FL 33131, USA,...","(25.7591323, -80.1918295, 0.0)",25.759132,-80.191829,0.0
9,1010 Brickell\r1010 Brickell Ave 2301\rMiami,3/29/2022,8/25/2022,118,"$1,499,000.00",1753,$855.11,Ana Teresa Rodriguez,Coldwell Banker Realty,David Jimenez PA,BHHS EWM Realty,1010 Brickell 1010 Brickell Ave 2301 Miami,1010 Brickell,1010 Brickell Avenue,"(1010 Brickell, 1010 Brickell Ave #2301, Miami...","(25.764055, -80.1923013, 0.0)",25.764055,-80.192301,0.0
10,S0outh Pointe Towers\r400 S Pointe Dr 906\rMia...,6/1/2022,8/25/2022,20,"$1,435,000.00",1042,"$1,377.16",John Lennon,South Pointe Drive Realty Inc,Adi Zilberberg,Gary Hennes Realtors,S0outh Pointe Towers 400 S Pointe Dr 906 Miami...,South Pointe Towers,400 S Pointe Drive,"(400 South Pointe Dr APT 906, Mi

Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

## Correction Section
To be used if a name is off. Format for making corrections is as follows:

`df.at[index_num,'column_name']=('what you want inserted')`

In [12]:
df.at[10,'building_name']=('South Pointe Towers')
df.at[10,'address_only']=('400 S Pointe Drive')

df.at[9,'building_name']=('1010 Brickell')
df.at[9,'address_only']=('1010 Brickell Avenue')

In [15]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=11)
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format('August 21st - August 27th')

# marker_cluster = MarkerCluster().add_to(m)

for i,r in df.iterrows():
    location = (r["lat"], r["lon"])
    info = (r["Sale Price"], r["days_on_market"], r["Agent"], r['Buyer Agent'], r['price_per_sqft'], r['address_only'], r['building_name'])
    info = list(info)
    new_line = '<br>'
    bold_start = '<strong>'
    bold_end = '</strong>'
    text = f'Building Name: {bold_start}{info[-1]}{bold_end}{new_line} \
    Sale Price: {bold_start}{info[0]}{bold_end}{new_line}Days on Market: \
    {bold_start}{info[1]}{bold_end}{new_line}Listing Agent: {bold_start}{info[2]} \
    {bold_end}{new_line}Buying Agent: {bold_start}{info[3]}{bold_end}{new_line}Price sq ft: \
    {bold_start}{info[4]}{bold_end}{new_line}Address: {bold_start}{info[5]}{bold_end}'
    folium.Marker(location=location,
                      tooltip=text)\
    .add_to(m)

m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))

m

In [16]:
m.save('index.html')

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [17]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [18]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/CondoSalesWeekEnding08_27_22
